In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-competition-fall2023/sample_submission.csv
/kaggle/input/final-competition-fall2023/foods_training.csv
/kaggle/input/final-competition-fall2023/foods_testing.csv


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [3]:
training_data = pd.read_csv("/kaggle/input/final-competition-fall2023/foods_training.csv", encoding = "latin1")
testing_data = pd.read_csv("/kaggle/input/final-competition-fall2023/foods_testing.csv", encoding = "latin1")

In [4]:
training_data['text'].fillna('', inplace=True)
testing_data['text'].fillna('', inplace=True)

In [5]:
X_train = training_data['text']
y_train = training_data['score_level']
X_test = testing_data['text']

In [6]:
tfidf_vectorizer = TfidfVectorizer(max_features= 10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [7]:
lr_model = LogisticRegression(max_iter = 10000)
lr_model.fit(X_train_tfidf,y_train)

LogisticRegression(max_iter=10000)

In [8]:
y_pred = lr_model.predict(X_test_tfidf)

In [9]:
predicted_labels = []
for score in y_pred:
    if score == 'high':
        predicted_labels.append('high')
    elif score == 'medium':
        predicted_labels.append('medium')
    elif score == 'low':
        predicted_labels.append('low')

In [10]:
submission_data = testing_data[['ID']]
#submission_data['score_level'] = predicted_labels[:len(testing_data)]
submission_data.loc[:, 'score_level'] = predicted_labels[:len(testing_data)]
subset_size = min(len(y_train), len(y_pred))
report = classification_report(y_train[:subset_size], y_pred[:subset_size])
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

        high       0.77      0.82      0.80      3866
         low       0.15      0.14      0.14       733
      medium       0.08      0.03      0.05       401

    accuracy                           0.66      5000
   macro avg       0.33      0.33      0.33      5000
weighted avg       0.63      0.66      0.64      5000



/tmp/ipykernel_20/1694582170.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_data.loc[:, 'score_level'] = predicted_labels[:len(testing_data)]


In [11]:
submission_data.to_csv("prediction_pk11.csv", index=False)